In [1]:
pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai langchain-chroma bs4

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress t

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) 
openai.api_key = os.environ['OPENAI_API_KEY']
os.environ['USER_AGENT'] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36"

In [2]:
pip install -qU langchain-openai


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo")

In [5]:
import bs4
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [11]:
for chunk in rag_chain.stream({"input" : "What is Task Decomposition?"}):
    print(chunk)

{'input': 'What is Task Decomposition?'}
{'context': [Document(page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}), Document(page_content='Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, 

In [10]:
for chunk in rag_chain.stream({"input" : "What is Task Decomposition?"}) :
    if answer_chunk := chunk.get("answer"):
        print(f"{answer_chunk}|",end = "")

Task| decomposition| involves| breaking| down| a| complex| task| into| smaller| and| more| manageable| steps|.| This| process| helps| agents| or| models| tackle| difficult| tasks| by| dividing| them| into| simpler| sub|tasks|.| Methods| like| Chain| of| Thought| and| Tree| of| Thoughts| are| used| to| guide| the| decomposition| process| and| enhance| performance| on| complex| tasks|.|

In [12]:
chain = rag_chain.pick("answer")

for chunk in chain.stream({"input" : "What is Task Decomposition?"}):
    print(f"{chunk}|",end = "")

|Task| decomposition| is| a| technique| used| to| break| down| complex| tasks| into| smaller| and| more| manageable| steps|.| It| involves| transforming| a| big| task| into| multiple| smaller| tasks| to| simplify| the| overall| process|.| This| approach| helps| agents| plan| ahead| and| tackle| intricate| tasks| effectively| by| focusing| on| individual| components| sequentially|.||

In [14]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question"
    "which might reference context in the chat history"
    "formulate a standalone question which can be understood"
    "without the chat history. Do NOT answer the question"
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human","{input}"),
    ]
)

contextualize_q_llm = llm.with_config(tags = ["contextualize_q_llm"])
history_aware_retriever = create_history_aware_retriever(
    contextualize_q_llm,retriever,contextualize_q_prompt
)


system_prompt = (
    "You are an assistant for question-answering tasks"
    "Use the following pieces of retrieved context to answer"
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human","{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm,qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever,question_answer_chain)

In [16]:
first_question = "What is Task Decomposition?"
first_answer = (
    "Task decomposition involves breaking up"
    "a complex task into smaller and simpler "
    "steps."
)

follow_up_question = "What are some common ways of doing it?"

chat_history = [
    ("human",first_question),
    ("ai",first_answer)
]

async for event in rag_chain.astream_events(
    {
        "input" : follow_up_question,
        "chat_history" : chat_history,
    },
    version = "v1",
):
    if(
        event["event"] == "on_chat_model_stream"
        and "contextualize_q_llm" in event["tags"]
    ):
        ai_message_chunk = event["data"]["chunk"]
        print(f"{ai_message_chunk.content}|",end = "")

|What| are| some| common| methods| used| for| task| decomposition|?||

In [18]:
async for event in rag_chain.astream_events(
    {
        "input" : follow_up_question,
        "chat_history" : chat_history,
    },
    version = "v1",
) : 
    if event["name"] == "Retriver":
        print(event)
        print()